# Code for basin delimitation

Developed by Rodrigo Aguayo (2020-2023)

In [ ]:
import grass.jupyter as gj
import grass.script as gs
from grass.pygrass.modules.grid import GridModule

from tqdm.notebook import tqdm

from geocube.vector import vectorize
import rioxarray as rioxr
import geopandas as gpd
import pandas as pd
import os

os.chdir('/home/rooda/Dropbox/Patagonia') 

## Elevation

In [ ]:
# import raster NASADEM 30 m
gs.run_command("r.import", input="GIS South/dem_patagonia1.tif", output="elevation_hr", flags = "o", overwrite=True)
gs.run_command("g.region", raster="elevation_hr", flags="p")

In [ ]:
# resample to ~90m
gs.run_command("g.region", raster="elevation_hr", res = 0.00083333)
gs.run_command('r.resamp.stats', input='elevation_hr', output='elevation_lr', overwrite = True)

In [ ]:
# sink removal using Lindsay et al. (2005) [MULTICORE]
grid = GridModule("r.hydrodem",
                  input="elevation_lr", 
                  output = "elevation_filled",
                  overwrite = True,
                  processes=20,
                  overlap=500)
grid.run()

## Delimitate all basins based of stream gauge location (intersection file)

In [ ]:
# calculate accumulation raster map and drainage direction raster map using MFD: multiple flow direction
gs.run_command("r.watershed", 
               elevation="elevation_filled", 
               threshold=500,
               drainage= "fdir", 
               accumulation="accum",
               overwrite = True)

# Performs stream network extraction [MULTICORE]
grid = GridModule("r.stream.extract",
                  elevation="elevation_filled",  
                  threshold=10000,
                  accumulation= "accum", 
                  stream_vector="stream_v", 
                  stream_raster="stream_r",
                  overwrite = True,
                  processes=8,
                  overlap=500)
grid.run()

In [ ]:
gs.run_command('r.out.gdal', input="accum", output=  'GIS South/accu_delete.tif', format='GTiff', overwrite=True)

In [ ]:
# stream gauge data
q_location = pd.read_csv("Data/Streamflow/Q_PMETobs_v10_metadata.csv").iloc[:,0:7]
q_location = gpd.GeoDataFrame(q_location, geometry= gpd.points_from_xy(x=q_location.gauge_lon, y=q_location.gauge_lat), crs="EPSG:4326")
q_location.to_file("GIS South/Basins_PMET_v10_points.shp")

gs.run_command("v.import",  input="GIS South/Basins_PMET_v10_points.shp", output="q_grass", flags = "o", overwrite=True)

In [ ]:
# Snap point to modelled stream network
gs.run_command("r.stream.snap", input="q_grass", output = "q_grass_snap", stream_rast="stream_r", radius = 50, overwrite=True)

# Delineates basins according stream network.
gs.run_command("r.stream.basins", flags="l", direction="fdir", points = "q_grass_snap", basins="basins", overwrite = True)
gs.run_command('r.out.gdal', input="basins", output=  'GIS South/Basins_PMET_v10_int.tif', format='GTiff', overwrite=True)  

In [ ]:
# from raster to shp
data = rioxr.open_rasterio("GIS South/Basins_PMET_v10_int.tif")
data.name = "gauge_id"
basins_int = vectorize(data)
basins_int = basins_int.set_crs(4326)
basins_int = basins_int.dissolve(by='gauge_id')
basins_int = basins_int.reset_index()

basins_int["gauge_id"]    = q_location.gauge_id
basins_int["gauge_name"]  = q_location.gauge_name
basins_int["institutio"] = q_location.institution
basins_int["int_area"]    = basins_int.to_crs(32719).area / 1e6

# check
print((basins_int.int_area > 10).sum() == len(basins_int))

basins_int.to_file("GIS South/Basins_PMETobs_int.shp")

## Delimitate all basins based of stream gauge location (1 polygon per basin)

In [ ]:
gs.run_command("v.out.ogr", input ="q_grass_snap", output = "GIS South/Basins_PMET_v10_points_s.shp", format = "ESRI_Shapefile", overwrite=True)
q_grass_snap = gpd.read_file("GIS South/Basins_PMET_v10_points_s.shp").get_coordinates()
q_grass_snap = q_grass_snap.set_index(basins_int.gauge_id)

In [ ]:
df = []

for basin in tqdm(q_grass_snap.index):
    coords = [q_grass_snap.x[basin], q_grass_snap.y[basin]]
    gs.run_command("r.water.outlet", input="fdir", output = "basin_i", coordinates = coords, overwrite=True)
    gs.run_command('r.out.gdal', input="basin_i", output=  'GIS South/Basins/Basin_{}.tif'.format(basin), format='GTiff', overwrite=True)
    data = rioxr.open_rasterio('GIS South/Basins/Basin_{}.tif'.format(basin))
    data.name = "gauge_id"
    data = vectorize(data)
    data["gauge_id"] = basin
    data = data.set_crs(4326)
    data = data.dissolve(by='gauge_id')
    data = data.reset_index()
    df.append(data)

df = pd.concat(df)
df = df.reset_index()

df["gauge_name"] = q_location.gauge_name
df["institutio"] = q_location.institution
df["int_area"]   = df.to_crs(32719).area / 1e6

df.to_file("GIS South/Basins_PMETobs.shp")